In [1]:
import time
import numpy as np
import random
import pandas as pd

In [2]:
def parseFile(file):
    time_start = time.time()

    content = []
    count, count_incomplete,count_complete, count_part = 0, 0, 0, 0

    with open(file) as txtfile:
        for row in txtfile:

            row = row.split(',')
            row[-1] = row[-1].strip()

            if count == 0:
                for i in range(len(row)):
                    row[i] = row[i].strip('"')

            content.append(row)

            count += 1

    content_mat = np.array(content)

    time_end = time.time()
    print('Reading data is complete! Running time is ' + str(time_end - time_start) + 's!')
    
    return content_mat

In [3]:
def filter_full_feature(mat):
    # remove all rows with missing value
    row_count = 0
    full_list = []
    for row in mat:
        if 'N/A' in row:
            pass
        else:
            full_list.append(row_count)

        row_count += 1

    mat = mat[full_list, :]
    return mat

In [4]:
def train_test_split(mat):
    train_list = []
    test_list = []
    num_sample, num_var = mat.shape

    for i in range(0, num_sample):
        if i == 0:
            train_list.append(i)
            test_list.append(i)
        else:
            rand = random.random()
            if rand >= 0.2:
                train_list.append(i)
            else:
                test_list.append(i)

    train_mat = mat[train_list, :]
    test_mat = mat[test_list, :]

    return train_mat, test_mat

In [7]:
def main():
    # this program is to calculate unemployment daily rate from gallup data
    
    content_mat = parseFile('CleanedData/gallup_clean_NA_determinant.txt') # no determinant, not filled
    num_sample, num_var = content_mat.shape # (3million,47)
    rate_feature_list = [6, 16, 21, 26] # sick, employ, age, date
    full_list = list(range(0, num_var))

    delete_list = list(set(full_list) - set(sentiment_list)) # index of all irrelavant features
    content_mat = np.delete(content_mat, delete_list, axis=1) # remove them

    content_mat = filter_full_feature(content_mat) # remove rows with missing values

    content_pd = pd.DataFrame(content_mat[1:, :], columns=content_mat[0,:]) # convert to pd dataframe

    # create tow lists
    unique_date = content_pd.intdate.unique() 
    num_date = unique_date.shape
    candidate_list = np.zeros(shape=num_date) # store total # of people with 20<=age<=65 and no sick
    unemployed_list = np.zeros(shape=num_date) # store # of people satisfying that condition but not employed

    for index, row in content_pd.iterrows():
        if int(row['age']) >= 20 and int(row['age']) <= 65 and int(row['healthprob_yes']) == 0:
            if int(row['employed']) == 1:
                idx = np.nonzero(unique_date == row['intdate'])
                candidate_list[idx] += 1
            else:
                idx = np.nonzero(unique_date == row['intdate'])
                candidate_list[idx] += 1
                unemployed_list[idx] += 1

    unemployed_rate = np.divide(unemployed_list, candidate_list)


    daily_employment_df = pd.DataFrame(np.transpose(unemployed_rate), index=np.transpose(unique_date), columns=['unemployed_rate'])
    daily_employment_df.to_csv(path_or_buf= 'CleanedData/daily_unemployment.csv')

if __name__ == "__main__":
    main()

Reading data is complete! Running time is 51.17492604255676s!
